Here we will:
1/ Extract the list of ETPs traded on six-group.com
2/ Remove tickers that give zero data on yahooFinance
3/ Remove precious metals ETPs
4/ Fetch crypto ETPs data 

In [76]:
import pandas as pd
import pandas_datareader as web
import numpy as np
from datetime import datetime
import seaborn as sns

from matplotlib import pyplot as plt
import matplotlib as matplot
import matplotlib.dates as mdates
import matplotlib.cbook as cbook
import requests
import json
from pandas.plotting import register_matplotlib_converters

#ETP csv file downloaded from six-group website to extract a list of ETP names & ISIN (Internation Security Identification Numbers)

df = pd.read_csv('ETP Explorer export_splittedintocolumns.csv')

#Preview our dataframe
df.head()

,FundLongName,ValorSymbol,FundReutersTicker,FundBloombergTicker,ValorNumber,ISIN,IssuerLongNameDesc,IssuerNameFull,TradingBaseCurrency,FundCurrency,...,DailyLowPrice,OnMarketVolume,OffBookVolume,TotalTurnover,TotalTurnoverCHF,ProductLineDesc,AssetClassDesc,UnderlyingGeographicalDesc,LegalStructureCountryDesc,UnderlyingProviderDesc
0,1 FiCAS Active Bitcoin Exchange Traded Product,BTCB,BTCB.S,BTCB SE,113027698,CH1130276988,Bitcoin Capital AG,Bitcoin Capital AG,CHF,CHF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1 FiCAS Active Ethereum Exchange Traded Product,ETHB,ETHB.S,ETHB SE,113027699,CH1130276996,Bitcoin Capital AG,Bitcoin Capital AG,CHF,CHF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,15 FiCAS Active Crypto ETP,BTCA,BTCA.S,BTCA SE,54868960,CH0548689600,Bitcoin Capital AG,Bitcoin Capital AG,CHF,CHF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,15 FiCAS Active Crypto ETP,BTCA,BTCA.S,BTCA SE,54868960,CH0548689600,Bitcoin Capital AG,Bitcoin Capital AG,EUR,CHF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,15 FiCAS Active Crypto ETP,BTCA,BTCA.S,BTCA SE,54868960,CH0548689600,Bitcoin Capital AG,Bitcoin Capital AG,USD,CHF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In order to have a list of inputs that is recognizable by Yahoo Finance API, we need to add a .SW to each line in the VALORSYMBOL column.

In [77]:
df['.SW'] = '.SW'
df['YahooTicker'] = df['ValorSymbol'] + df['.SW']

Checking the columns' names to know which columns to drop and keep.

In [78]:
print(df.columns)

Index(['FundLongName', 'ValorSymbol', 'FundReutersTicker',
       'FundBloombergTicker', 'ValorNumber', 'ISIN', 'IssuerLongNameDesc',
       'IssuerNameFull', 'TradingBaseCurrency', 'FundCurrency',
       'ManagementFee', 'ReplicationMethodDesc', 'ManagementStyleDesc',
       'MarketMakers', 'ClosingPrice', 'ClosingPerformance', 'ClosingDelta',
       'FundUnderlyingDescription', 'BidVolume', 'BidPrice', 'AskVolume',
       'AskPrice', 'MidSpread', 'PreviousClosingPrice', 'MarketDate',
       'MarketTime', 'OpeningPrice', 'OpeningPrice.1', 'DailyLowPrice',
       'OnMarketVolume', 'OffBookVolume', 'TotalTurnover', 'TotalTurnoverCHF',
       'ProductLineDesc', 'AssetClassDesc', 'UnderlyingGeographicalDesc',
       'LegalStructureCountryDesc', 'UnderlyingProviderDesc', '.SW',
       'YahooTicker'],
      dtype='object')


If we fetch the data from yahooFinance, we care about the columns 'YahooTicker', 'FundLongName', 'ISIN', 'FundCurrency's,'FundUnderlyingDecription'

In [79]:
#Dropping unnecessary columns

df.drop(columns = ['FundReutersTicker',
       'FundBloombergTicker', 'ValorNumber', 'IssuerLongNameDesc',
       'IssuerNameFull', 'TradingBaseCurrency', 
       'ManagementFee', 'ReplicationMethodDesc', 'ManagementStyleDesc',
       'MarketMakers', 'ClosingPrice', 'ClosingPerformance', 'ClosingDelta',
       'BidVolume', 'BidPrice', 'AskVolume',
       'AskPrice', 'MidSpread', 'PreviousClosingPrice', 'MarketDate',
       'MarketTime', 'OpeningPrice', 'OpeningPrice.1', 'DailyLowPrice',
       'OnMarketVolume', 'OffBookVolume', 'TotalTurnover', 'TotalTurnoverCHF',
       'ProductLineDesc', 'AssetClassDesc', 'UnderlyingGeographicalDesc',
       'LegalStructureCountryDesc', 'UnderlyingProviderDesc'], axis = 1, inplace = True)

#Option to view all rows to check for errors in YahooTicker column
pd.set_option('display.max_rows', None)
df

,FundLongName,ValorSymbol,ISIN,FundCurrency,FundUnderlyingDescription,.SW,YahooTicker
0,1 FiCAS Active Bitcoin Exchange Traded Product,BTCB,CH1130276988,CHF,Bitcoin (BTC),.SW,BTCB.SW
1,1 FiCAS Active Ethereum Exchange Traded Product,ETHB,CH1130276996,CHF,Ethereum (ETH),.SW,ETHB.SW
2,15 FiCAS Active Crypto ETP,BTCA,CH0548689600,CHF,top 15 digital assets,.SW,BTCA.SW
3,15 FiCAS Active Crypto ETP,BTCA,CH0548689600,CHF,top 15 digital assets,.SW,BTCA.SW
4,15 FiCAS Active Crypto ETP,BTCA,CH0548689600,CHF,top 15 digital assets,.SW,BTCA.SW
5,21Shares Algorand (ALGO) ETP,ALGO,CH1146882316,USD,NaN,.SW,ALGO.SW
6,21Shares Algorand (ALGO) ETP,ALGO,CH1146882316,USD,NaN,.SW,ALGO.SW
7,21Shares Algorand (ALGO) ETP,ALGO,CH1146882316,USD,NaN,.SW,ALGO.SW
8,21Shares Avalanche (AVAX) ETP,AVAX,CH1135202088,USD,NaN,.SW,AVAX.SW
9,21Shares Avalanche (AVAX) ETP,AVAX,CH1135202088,USD,NaN,.SW,AVAX.SW


Next we will import the list of ETP that give us errors, using this list we will remove the rows containing the faulty tickers + precious metals ETPs tickers from our initial DF in order to fetch data from yahoo finance with issues.

In [80]:
#Removing tickers that give 0 data from yahooFinance
errors_tickers_list = pd.read_csv('errors_list.csv')
errors_tickers_list.drop(['Unnamed: 0'], axis=1, inplace=True)
errors_tickers = errors_tickers_list['0'].tolist()
print("The following are ETP tickers that give 0 data on yahooFin: ", errors_tickers)

df = df[~df.ValorSymbol.isin(errors_tickers)]

#Removing precious metals ETP from list 
metals_list = [ 'XGLD', 'XAD1', 'XAD2' ]
df = df[~df.ValorSymbol.isin(metals_list)]


The following are ETP tickers that give 0 data on yahooFin:  ['TPLT', 'WALT', 'TNIK', 'AVAX', 'TSLV', 'PLKA', 'BTIC', 'VAVA', 'LUNA', 'LITE', 'VBTC', 'RDAN', 'ALTS', 'TCOP', 'VETH', 'ADOT', 'ESOL', 'BTCE', 'VTOP', 'POLY', 'VDOT', 'SGLE', 'ELTC', 'VSOL', 'BLOC', 'TPAL', 'ATOM', 'TGLD', 'XBTI', 'ZETH', 'MEGA', 'ALGO']


In [81]:
#Checkikng our DF so far
df

,FundLongName,ValorSymbol,ISIN,FundCurrency,FundUnderlyingDescription,.SW,YahooTicker
0,1 FiCAS Active Bitcoin Exchange Traded Product,BTCB,CH1130276988,CHF,Bitcoin (BTC),.SW,BTCB.SW
1,1 FiCAS Active Ethereum Exchange Traded Product,ETHB,CH1130276996,CHF,Ethereum (ETH),.SW,ETHB.SW
2,15 FiCAS Active Crypto ETP,BTCA,CH0548689600,CHF,top 15 digital assets,.SW,BTCA.SW
3,15 FiCAS Active Crypto ETP,BTCA,CH0548689600,CHF,top 15 digital assets,.SW,BTCA.SW
4,15 FiCAS Active Crypto ETP,BTCA,CH0548689600,CHF,top 15 digital assets,.SW,BTCA.SW
11,21Shares Binance BNB ETP,ABNB,CH0496454155,USD,N/A,.SW,ABNB.SW
12,21Shares Bitcoin Cash ETP,ABCH,CH0475552201,USD,Bitcoin Cash,.SW,ABCH.SW
13,21Shares Bitcoin ETP,ABTC,CH0454664001,USD,Bitcoin,.SW,ABTC.SW
14,21Shares Bitcoin ETP,ABTC,CH0454664001,USD,Bitcoin,.SW,ABTC.SW
15,21Shares Bitcoin ETP,ABTC,CH0454664001,USD,Bitcoin,.SW,ABTC.SW


We will rely on a dictionary with YahooTicker as key values for data fetching from Yahoo Finance.
And we will also collect BTC data to add to our ETP database for analysis purpose.

Dictionary Creation, YahooTicker in keys & FundLongName in values.

In [82]:
#Creating our dicrtionary then assigning FundLongName and YahooTicker to a dictionary
#First assigning each column to a list 

ETP_dict = dict(zip(df.YahooTicker, df.FundLongName))

#Uncomment below line to preview of our dictionary
print(ETP_dict)

{'BTCB.SW': '1 FiCAS Active Bitcoin Exchange Traded Product', 'ETHB.SW': '1 FiCAS Active Ethereum Exchange Traded Product', 'BTCA.SW': '15 FiCAS Active Crypto ETP', 'ABNB.SW': '21Shares Binance BNB ETP', 'ABCH.SW': '21Shares Bitcoin Cash ETP', 'ABTC.SW': '21Shares Bitcoin ETP', 'ABBA.SW': '21Shares Bitcoin Suisse Index ETP', 'KEYS.SW': '21Shares Bitwise Select 10 Large Cap Crypto Index ETP', 'AADA.SW': '21Shares Cardano ETP', 'HODLX.SW': '21Shares Crypto Basket 10 ETP', 'HODLV.SW': '21Shares Crypto Basket Equal Weight ETP', 'HODL.SW': '21Shares Crypto Basket Index', 'AETH.SW': '21Shares Ethereum ETP', 'AXRP.SW': '21Shares Ripple XRP ETP', 'SBTC.SW': '21Shares Short Bitcoin ETP', 'ASOL.SW': '21Shares Solana (SOL) ETP', 'AXLM.SW': '21Shares Stellar ETP', 'AXTZ.SW': '21Shares Tezos ETP', 'BITC.SW': 'COINSHARES PHYSICAL BITCOIN (BTC)', 'ETHE.SW': 'COINSHARES PHYSICAL ETHEREUM (ETH)', 'XRPL.SW': 'COINSHARES PHYSICAL XRP', 'SGLD.SW': 'Invesco Physical Gold ETC', 'SBTCC.SW': 'SEBA Bitcoin ETP

In [94]:
#Collecting BTC daily data from yahoo finance
import yfinance as yf

#Setting today's date
today_date = datetime.today().strftime("%Y-%m-%d")
btc_df = yf.download("BTC-USD", start = "2020-01-01", end = today_date)
btc_df.drop(columns= ['Adj Close'], axis = 1, inplace = True)
btc_df.rename(columns= {"Open":"BTC-Open", "Close":"BTC-Close", "Volume" : "BTC-Volume"}, inplace = True)

delta = btc_df['BTC-Open'] - btc_df['BTC-Close']
btc_df['Delta'] = delta 
btc_df.head()

[*********************100%***********************]  1 of 1 completed


,BTC-Open,High,Low,BTC-Close,BTC-Volume,Delta
Date,,,,,,
2020-01-01,7194.892090,7254.330566,7174.944336,7200.174316,18565664997,-5.282227
2020-01-02,7202.551270,7212.155273,6935.270020,6985.470215,20802083465,217.081055
2020-01-03,6984.428711,7413.715332,6914.996094,7344.884277,28111481032,-360.455566
2020-01-04,7345.375488,7427.385742,7309.514160,7410.656738,18444271275,-65.281250
2020-01-05,7410.451660,7544.497070,7400.535645,7411.317383,19725074095,-0.865723


In [95]:
#Given the random nature of variations between OPEN & CLOSE we expect some negative Delta Hr Trend values 
#Verification of negative values existences
(btc_df < 0).any().any()

True

In [96]:
# pip install yfinance --upgrade --no-cache-dir

In [97]:
print(ETP_df.columns)

Index([               'BTC-Open',                'BTC-High',
                       'BTC-Low',               'BTC-Close',
                    'BTC-Volume',                   'Delta',
        ('Adj Close', 'AADA.SW'),  ('Adj Close', 'ABBA.SW'),
        ('Adj Close', 'ABCH.SW'),  ('Adj Close', 'ABNB.SW'),
        ('Adj Close', 'ABTC.SW'),  ('Adj Close', 'AETH.SW'),
        ('Adj Close', 'ASOL.SW'),  ('Adj Close', 'AXLM.SW'),
        ('Adj Close', 'AXRP.SW'),  ('Adj Close', 'AXTZ.SW'),
        ('Adj Close', 'BITC.SW'),  ('Adj Close', 'BTCA.SW'),
        ('Adj Close', 'BTCB.SW'),  ('Adj Close', 'BTCW.SW'),
        ('Adj Close', 'ETHB.SW'),  ('Adj Close', 'ETHE.SW'),
        ('Adj Close', 'ETHW.SW'),  ('Adj Close', 'HODL.SW'),
       ('Adj Close', 'HODLV.SW'), ('Adj Close', 'HODLX.SW'),
        ('Adj Close', 'KEYS.SW'),  ('Adj Close', 'MOON.SW'),
        ('Adj Close', 'SBTC.SW'), ('Adj Close', 'SBTCC.SW'),
       ('Adj Close', 'SBTCU.SW'),  ('Adj Close', 'SDOT.SW'),
       ('Adj Close', 'SE

In [98]:
#Collecting data for ETPs
#Declaring ETPs 
list = ETP_dict.keys()
# print(list)
list_ETPs = 'BTCB.SW', 'ETHB.SW', 'BTCA.SW', 'ABNB.SW', 'ABCH.SW', 'ABTC.SW', 'ABBA.SW', 'KEYS.SW', 'AADA.SW', 'HODLX.SW', 'HODLV.SW', 'HODL.SW', 'AETH.SW', 'AXRP.SW', 'SBTC.SW', 'ASOL.SW', 'AXLM.SW', 'AXTZ.SW', 'BITC.SW', 'ETHE.SW', 'XRPL.SW', 'SGLD.SW', 'SBTCC.SW', 'SBTCU.SW', 'SEBAX.SW', 'SETH.SW', 'SDOT.SW', 'MOON.SW', 'VPOL.SW', 'BTCW.SW', 'ETHW.SW'


ETP_temp = yf.download(list_ETPs, start = "2020-01-01", end = "2022-02-18", data_source = "yahoo")
ETP_temp.drop(columns = ['Volume', 'Open', 'Close','Low', 'High'], inplace= True)
ETP_temp.head()

#Merging with btc_data
ETP_df = pd.merge(btc_df, ETP_temp, how = 'right', on =['Date', 'Date'])

ETP_df.head()

[*********************100%***********************]  31 of 31 completed


C:\Users\Fluffy\AppData\Local\Temp/ipykernel_3972/1516704809.py:13: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left,2 on the right)
  ETP_df = pd.merge(btc_df, ETP_temp, how = 'right', on =['Date', 'Date'])


,,BTC-Open,High,Low,BTC-Close,BTC-Volume,Delta,"(Adj Close, AADA.SW)","(Adj Close, ABBA.SW)","(Adj Close, ABCH.SW)","(Adj Close, ABNB.SW)",...,"(Adj Close, MOON.SW)","(Adj Close, SBTC.SW)","(Adj Close, SBTCC.SW)","(Adj Close, SBTCU.SW)","(Adj Close, SDOT.SW)","(Adj Close, SEBAX.SW)","(Adj Close, SETH.SW)","(Adj Close, SGLD.SW)","(Adj Close, VPOL.SW)","(Adj Close, XRPL.SW)"
Date,Date,,,,,,,,,,,,,,,,,,,,,
2020-01-03,2020-01-03,6984.428711,7413.715332,6914.996094,7344.884277,28111481032,-360.455566,NaN,16.931999,10.286,15.010000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,150.220001,NaN,NaN
2020-01-06,2020-01-06,7410.452148,7781.867188,7409.292969,7769.219238,23276261598,-358.767090,NaN,17.350000,11.220,15.970000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,152.759995,NaN,NaN
2020-01-07,2020-01-07,7768.682129,8178.215820,7768.227539,8163.692383,28767291327,-395.010254,NaN,18.129999,11.228,16.059999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,152.139999,NaN,NaN
2020-01-08,2020-01-08,8161.935547,8396.738281,7956.774414,8079.862793,31672559265,82.072754,NaN,19.049999,11.572,16.059999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,152.139999,NaN,NaN
2020-01-09,2020-01-09,8082.295898,8082.295898,7842.403809,7879.071289,24045990466,203.224609,NaN,18.020000,11.130,15.850000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,150.080002,NaN,NaN


As you may notice some ETPs like AADA, SBTC, SBTCC have launched after our start date , January 03 2020.
The launch dates vary for each ETP but we will focus on data since the onset of COVID January 01 2020.

In [103]:
#Let's have a look at the whole dataset.
pd.set_option('display.max_rows', None)

#Renaming our final dataset
ETP_BTC_df = ETP_df.copy()
ETP_BTC_df.to_csv('ETP_BTC_df.csv')

ETP_BTC_df


,,BTC-Open,High,Low,BTC-Close,BTC-Volume,Delta,"(Adj Close, AADA.SW)","(Adj Close, ABBA.SW)","(Adj Close, ABCH.SW)","(Adj Close, ABNB.SW)",...,"(Adj Close, MOON.SW)","(Adj Close, SBTC.SW)","(Adj Close, SBTCC.SW)","(Adj Close, SBTCU.SW)","(Adj Close, SDOT.SW)","(Adj Close, SEBAX.SW)","(Adj Close, SETH.SW)","(Adj Close, SGLD.SW)","(Adj Close, VPOL.SW)","(Adj Close, XRPL.SW)"
Date,Date,,,,,,,,,,,,,,,,,,,,,
2020-01-03,2020-01-03,6984.428711,7413.715332,6914.996094,7344.884277,28111481032,-360.455566,NaN,16.931999,10.286000,15.010000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,150.220001,NaN,NaN
2020-01-06,2020-01-06,7410.452148,7781.867188,7409.292969,7769.219238,23276261598,-358.767090,NaN,17.350000,11.220000,15.970000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,152.759995,NaN,NaN
2020-01-07,2020-01-07,7768.682129,8178.215820,7768.227539,8163.692383,28767291327,-395.010254,NaN,18.129999,11.228000,16.059999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,152.139999,NaN,NaN
2020-01-08,2020-01-08,8161.935547,8396.738281,7956.774414,8079.862793,31672559265,82.072754,NaN,19.049999,11.572000,16.059999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,152.139999,NaN,NaN
2020-01-09,2020-01-09,8082.295898,8082.295898,7842.403809,7879.071289,24045990466,203.224609,NaN,18.020000,11.130000,15.850000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,150.080002,NaN,NaN
2020-01-10,2020-01-10,7878.307617,8166.554199,7726.774902,8166.554199,28714583844,-288.246582,NaN,18.500000,11.850000,16.160000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,151.399994,NaN,NaN
2020-01-13,2020-01-13,8189.771973,8197.788086,8079.700684,8144.194336,22482910688,45.577637,NaN,18.500000,12.432000,16.620001,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,150.740005,NaN,NaN
2020-01-14,2020-01-14,8140.933105,8879.511719,8140.933105,8827.764648,44841784107,-686.831543,NaN,20.040001,16.316000,17.290001,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,149.720001,NaN,NaN
2020-01-15,2020-01-15,8825.343750,8890.117188,8657.187500,8807.010742,40102834650,18.333008,NaN,20.200001,16.500000,17.290001,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,150.600006,NaN,NaN
